<a href="https://colab.research.google.com/github/yebiny/SkillTreePython-DeepLearning/blob/main/04.Paper-with-code/commet/ch04_07_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ch04.07 Variational AutoEncoder

* 날짜:
* 이름:

## 학습내용
    - Variational AutoEncoder를 이해하고 구현한다.


In [1]:
import sys, os
sys.path.append('/content/drive/MyDrive/scripts/')
from lib import *

## VAE 기본 구현
---

![](https://user-images.githubusercontent.com/24144491/50323466-18d03700-051d-11e9-82ed-afb1b6e2666a.png)

### **| Encoder, Decoder 구현**

In [20]:
class Sampling(layers.Layer): # 상속

  def call(self, inputs):
    z_mu, z_sigma = inputs
    z_epsilon = tf.keras.backend.random_normal(shape=tf.shape(z_mu))
    z = ( tf.exp(z_epsilon*0.5) * z_sigma ) + z_mu
    return z

class BuildModel():
  def __init__(self, x_shape=(28,28,1), z_dims=100):
    self.x_shape = x_shape
    self.z_dims = z_dims

  def build_encoder(self):
    x = layers.Input(shape = self.x_shape )
    z = layers.Conv2D(32, 3, strides=2, padding='same', activation='leaky_relu')(x)
    z = layers.Conv2D(32, 3, strides=2, padding='same', activation='leaky_relu')(z)
    self.z_conv_shape = z.shape[1:] # (7, 7, 32)
 
    z = layers.Flatten()(z)
    z_mu = layers.Dense(z_dims, activation='relu')(z)
    z_sigma = layers.Dense(z_dims, activation='relu')(z)

    z = Sampling()([z_mu, z_sigma])
    encoder = models.Model(x, [z_mu, z_sigma, z], name='Encoder')
    return encoder

  def build_decoder(self):
    z = layers.Input(shape = self.z_dims)
    y = layers.Dense(256, activation='relu')(z)
    dim = self.z_conv_shape[0]*self.z_conv_shape[1]*self.z_conv_shape[2]
    y = layers.Dense(dim, activation = 'relu')(y)
    y = layers.Reshape(self.z_conv_shape)(y)
    y = layers.Conv2DTranspose(32, 3, strides=2, padding='same', activation='leaky_relu')(y)
    y = layers.Conv2DTranspose(1, 3, strides=2, padding='same', activation='sigmoid')(y)

    decoder = models.Model(z, y, name='Decoder')
    return decoder

In [21]:
x_shape = (28,28,1)
z_dims = 10
builder = BuildModel(x_shape, z_dims)
encoder = builder.build_encoder()
decoder = builder.build_decoder()
encoder.summary()
decoder.summary()

Model: "Encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_18 (Conv2D)             (None, 14, 14, 32)   320         ['input_14[0][0]']               
                                                                                                  
 conv2d_19 (Conv2D)             (None, 7, 7, 32)     9248        ['conv2d_18[0][0]']              
                                                                                                  
 flatten_9 (Flatten)            (None, 1568)         0           ['conv2d_19[0][0]']              
                                                                                            

### **| 손실함수 정의**

![](https://user-images.githubusercontent.com/24144491/50323472-1a016400-051d-11e9-86b7-d8bf6a1a880f.png)

In [23]:
'''loss.py'''
def get_rec_loss(imgs, recs):
  rec_loss = tf.keras.losses.binary_crossentropy(imgs, recs)
  rec_loss = tf.reduce_mean(rec_loss)
  return rec_loss

def get_kl_loss(z_mu, z_sigma):
  kl_loss = tf.square(z_mu) + ( -tf.exp(z_sigma) + z_sigma + 1)
  kl_loss = -0.5*tf.reduce_mean(kl_loss)
  return kl_loss

### **| VAE 구현**

In [40]:
class VAE():
  def __init__(self, encoder, decoder, x_shape):
    self.encoder = encoder
    self.decoder = decoder
    self.x_shape = x_shape

  def compile(self, optimizer = tf.keras.optimizers.Adam() ):
    img = layers.Input(shape = x_shape)
    z_mu, z_sigma, z  = self.encoder(img)
    rec = self.decoder(z)

    self.vae = models.Model(img, rec, name='VAE')
    self.optimizer = optimizer

  @tf.function
  def train_step(self, imgs):
    # imgs.shape = (16, 28, 28, 1)
    with tf.GradientTape() as tape:
      
      # propagation
      z_mu, z_sigma, z = self.encoder(imgs)
      recs = self.decoder(z)

      # loss 계산 - (rec loss / kl loss)
      rec_loss = get_rec_loss(imgs, recs) # 원본 이미지와 재구성한 이미지가 최대한 비슷해지면 좋겠다
      kl_loss = get_kl_loss(z_mu, z_sigma) # z가 다양하면서, 정보량 유사
      loss = rec_loss + kl_loss

    # gradient 계산
    weights = self.vae.trainable_variables
    gradients = tape.gradient(loss, weights)

    # weight 업데이트
    self.optimizer.apply_gradients(zip(gradients,weights))

    return loss, rec_loss, kl_loss

  def fit(self, x_data, epochs = 1, batch_size=16):

    train_ds = tf.data.Dataset.from_tensor_slices(x_data).batch(batch_size)
    self.batch_size=batch_size
    history = {'loss':[0 for i in range(epochs)], 
               'rec_loss':[0 for i in range(epochs)], 
               'kl_loss':[0 for i in range(epochs)] }

    for epoch in range(1, epochs+1):
      self.epoch=epoch     
      for imgs in train_ds:
        loss, rec_loss, kl_loss = self.train_step(imgs)
        history['loss'][epoch-1]+=loss
        history['rec_loss'][epoch-1]+=rec_loss
        history['kl_loss'][epoch-1]+=kl_loss
      print(history)

### **| 학습**

In [41]:
def get_mnist():
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
  x_train = x_train.astype(np.float32) / 255
  x_train = x_train.reshape(-1, 28, 28, 1)
  x_test = x_test.astype(np.float32) / 255
  x_test = x_test.reshape(-1, 28, 28, 1)

  return x_train, x_test

In [42]:
x_train, x_test = get_mnist()
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [43]:
x_shape = (28,28,1)
z_dims = 100
builder = BuildModel(x_shape, z_dims)
encoder = builder.build_encoder()
decoder = builder.build_decoder()

vae = VAE(encoder, decoder, x_shape)
vae.compile()
vae.vae.summary()

Model: "VAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 Encoder (Functional)        [(None, 100),             323368    
                              (None, 100),                       
                              (None, 100)]                       
                                                                 
 Decoder (Functional)        (None, 28, 28, 1)         438369    
                                                                 
Total params: 761,737
Trainable params: 761,737
Non-trainable params: 0
_________________________________________________________________


In [ ]:
vae.fit(x_train, 5)

### **| 결과 확인**